In [ ]:
include("src/Sim.jl")

In [ ]:
using DifferentialEquations
using Plots
using .Sim

In [ ]:
@macroexpand @model RLC begin
    @parameter R L C
    @blk int1 = IntegratorBlock() inport[4]:x
    @blk int2 = IntegratorBlock()
    @blk in1 = InBlock()
    @blk out1 = OutBlock()
    @blk gain1 = GainBlock(R)
    @blk gain2 = GainBlock(1/C)
    @blk gain3 = GainBlock(1/L)
    @connect gain3 <= exp(in1.outport - gain1 - int1)
    @connect int2 <= gain3
    @connect out1 <= int2
    @connect gain1 <= int2
    @connect gain2 <= int2
    @connect int1 <= gain2
end

In [ ]:
@model RLC begin
    @parameter R L C
    @blk int1 = IntegratorBlock()
    @blk int2 = IntegratorBlock()
    @blk in1 = InBlock()
    @blk out1 = OutBlock()
    @blk gain1 = GainBlock(R)
    @blk gain2 = GainBlock(1/C)
    @blk gain3 = GainBlock(1/L)
    @connect gain3 <= in1 - gain1 - int1
    @connect int2 <= gain3
    @connect out1 <= int2
    @connect gain1 <= int2
    @connect gain2 <= int2
    @connect int1 <= gain2
end

In [ ]:
systemequation(RLC())

In [ ]:
@model Test begin
    @parameter R L C voltage
    @blk system = RLC(R=R, L=L, C=C) outport:i
    @blk source = StepBlock(steptime=0.1, finalvalue=voltage) outport:v
    @connect source => system
end

In [ ]:
m = Test(R=10, L=100e-3, C=100e-6, voltage=5)
sol = @simulate(m, tspan = (0.0, 1.0), scope=(i, v))
sol.graph